# Chapter 3 スクレイピングでデータを収集しよう

## Lesson 21 Webページを取得してみましょう

### 手順パート: 書籍ページのHTMLを取得する

『いちばんやさしいPythonの教本』の書籍ページを取得する

In [1]:
import requests  # requestsをインポート
res = requests.get('https://book.impress.co.jp/books/1116101151')  # 書籍ページのURLを指定して、レスポンスを取得
res.status_code  # ステータスコードを確認

200

In [2]:
html_doc = res.text
print(html_doc[:150])








<!DOCTYPE html>
<html lang="ja" dir="ltr">
<head>
<meta charset="utf-8" />

<title>いちばんやさしいPythonの教本 人気講師が教える基礎からサーバサイド開発まで - インプレスブックス</title>


## Lesson 22 Webページをスクレイピングしましょう

In [3]:
from bs4 import BeautifulSoup

html_doc = '''
<html>
    <head>
        <title>いちやさサンプル</title>
    </head>
    <body>
        <h1 class="not_link">スクレイピング用サンプル</h1>
        <ul id="first_ul">
            <li><a href="http://ichiyasa.sample/link1">リンク1</a></li>
            <li><a href="http://ichiyasa.sample/link2">リンク2</a></li>
            <li class="not_link">テキスト1</li>
        </ul>
        <ul id="second_ul">
            <li><a href="http://ichiyasa.sample/link3">リンク3</a></li>
            <li class="not_link">テキスト2</li>
        </ul>
    </body>
</html>
'''

soup = BeautifulSoup(html_doc, 'html.parser')

In [6]:
a_tag = soup.find('a')  # 最初のa要素を検索
print('内容:', a_tag.get_text())  # 要素の内容を取得
print('href属性:', a_tag['href'])  # 要素の属性値を取得

内容: リンク1
href属性: http://ichiyasa.sample/link1


In [6]:
# id属性で検索
soup.find(id='second_ul')

<ul id="second_ul">
<li><a href="http://ichiyasa.sample/link3">リンク3</a></li>
<li class="not_link">テキスト2</li>
</ul>

In [7]:
# class属性で検索
soup.find(class_='not_link')

<h1 class="not_link">スクレイピング用サンプル</h1>

In [8]:
# 要素名とclass属性で検索
soup.find('li', class_='not_link')

<li class="not_link">テキスト1</li>

In [9]:
# テキスト2のli要素を検索
ul_tag = soup.find('ul', id='second_ul')
ul_tag.find('li', class_='not_link')

<li class="not_link">テキスト2</li>

In [10]:
# 全てのa要素を取得
a_tags = soup.find_all('a')
for a_tag in a_tags:
    print(a_tag)

<a href="http://ichiyasa.sample/link1">リンク1</a>
<a href="http://ichiyasa.sample/link2">リンク2</a>
<a href="http://ichiyasa.sample/link3">リンク3</a>


In [11]:
# 複数の要素名で検索
multi_tags = soup.find_all(['h1', 'a'])
for tag in multi_tags:
    print(tag)

<h1 class="not_link">スクレイピング用サンプル</h1>
<a href="http://ichiyasa.sample/link1">リンク1</a>
<a href="http://ichiyasa.sample/link2">リンク2</a>
<a href="http://ichiyasa.sample/link3">リンク3</a>


### 手順パート: 書籍ページから書籍名と値段を取得する

In [9]:
import requests
from bs4 import BeautifulSoup
res = requests.get('https://book.impress.co.jp/books/1116101151')
html_doc = res.text
soup = BeautifulSoup(html_doc, 'html.parser')

In [11]:
div_book_detail = soup.find('div', class_='block-book-detail')
div_book_detail

<div class="block-book-detail">
<div class="block-book-detail-head">
<h2>いちばんやさしいPythonの教本 人気講師が教える基礎からサーバサイド開発まで</h2>
</div>
<div class="block-book-detail-img">
<div class="module-book-img">
<p><img alt="いちばんやさしいPythonの教本 人気講師が教える基礎からサーバサイド開発まで" src="//img.ips.co.jp/ij/16/1116101151/1116101151-520x.jpg" width="260"/></p>
</div>
<div class="module-book-info">
<p class="module-book-price">2,420円<span class="module-price-note">（本体 2,200円＋税10%）</span></p>
<dl class="module-book-data">
<dt>品種名</dt><dd>書籍</dd>
<dt>発売日</dt><dd>2017/8/10</dd>
<dt>ページ数</dt><dd>272</dd>
<dt>サイズ</dt><dd>B5変形判</dd>
<dt>著者</dt><dd>
鈴木 たかのり　著/杉谷 弥月　著/株式会社ビープラウド　著
</dd>
<dt>ISBN</dt><dd>
9784295002086</dd>
</dl>
</div>
<!--//block-book-detail-img--></div>
<div class="block-book-detail-body">
<div class="module-book-link-tab">
<ul class="module-inline-block"><li class="module-otameshi"><a href="http://impress.tameshiyo.me/9784295002086" target="_blank">試し読み</a></li><li class="module-download"><a href="#box-download">

In [8]:
book_title = div_book_detail.find('h2')
book_title.get_text()

'いちばんやさしいPythonの教本 人気講師が教える基礎からサーバサイド開発まで'

In [14]:
book_price = div_book_detail.find('p', class_='module-book-price')
book_price.get_text()


'2,420円（本体 2,200円＋税10%）'

## Lesson 23 少し難しいスクレイピングに挑戦しましょう

In [12]:
html_doc = '''
<dl id='python-books'>
  <dt>Python入門本</dt><dd>1000円</dd>
  <dt>スクレイピング本</dt><dd>1500円</dd>
  <dt>機械学習本</dt><dd>2000円</dd>
</dl>
'''

In [13]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html_doc, 'html.parser')
python_books = {}
for tag in soup.find_all(['dt', 'dd']):
    if tag.name == 'dt':  # dt要素の場合はキー
        key = tag.get_text()
    if tag.name  == 'dd':  # dd要素の場合は値
        python_books[key] = tag.get_text()

print('Python本の辞書: ', python_books)
print('スクレイピング本の値段: ', python_books['スクレイピング本'])

Python本の辞書:  {'Python入門本': '1000円', 'スクレイピング本': '1500円', '機械学習本': '2000円'}
スクレイピング本の値段:  1500円


### 手順パート: 書籍ページから発売日と著者を取得する

In [18]:
import requests
from bs4 import BeautifulSoup

res = requests.get('https://book.impress.co.jp/books/1116101151')
html_doc = res.text
soup = BeautifulSoup(html_doc, 'html.parser')
div_book_detail = soup.find('div', class_='block-book-detail')

In [19]:
dl_book_data = div_book_detail.find('dl', class_='module-book-data')
dl_book_data

<dl class="module-book-data">
<dt>品種名</dt><dd>書籍</dd>
<dt>発売日</dt><dd>2017/8/10</dd>
<dt>ページ数</dt><dd>272</dd>
<dt>サイズ</dt><dd>B5変形判</dd>
<dt>著者</dt><dd>
鈴木 たかのり　著/杉谷 弥月　著/株式会社ビープラウド　著
</dd>
<dt>ISBN</dt><dd>
9784295002086</dd>
</dl>

In [20]:
book_data = {}
for tag in dl_book_data.find_all(['dt', 'dd']):
    if tag.name == 'dt':
        key = tag.get_text()
    if tag.name  == 'dd':
        book_data[key] = tag.get_text().strip()
        
book_data

{'品種名': '書籍',
 '発売日': '2017/8/10',
 'ページ数': '272',
 'サイズ': 'B5変形判',
 '著者': '鈴木 たかのり\u3000著/杉谷 弥月\u3000著/株式会社ビープラウド\u3000著',
 'ISBN': '9784295002086'}

In [21]:
print('発売日:', book_data['発売日'])
print('著者:', book_data['著者'])

発売日: 2017/8/10
著者: 鈴木 たかのり　著/杉谷 弥月　著/株式会社ビープラウド　著


## Lesson 24 複数のWebページからデータを集めましょう

### 手順パート: 複数の書籍ページをスクレイピングする

In [46]:
import requests
from bs4 import BeautifulSoup

res = requests.get('https://book.impress.co.jp/booklist/')
html_doc = res.text
soup = BeautifulSoup(html_doc, 'html.parser')
div_book_list = soup.find('div', class_='block-book-list-body')

In [47]:
book_urls = []  # 書籍ページのURLリスト
a_tags = div_book_list.find_all('a')
for a_tag in a_tags:
    if a_tag['href'] not in book_urls:  # 重複しないように存在チェック
        book_urls.append(a_tag['href'])
book_urls

['https://book.impress.co.jp/books/1121101107',
 'https://book.impress.co.jp/books/1122101010',
 'https://book.impress.co.jp/books/1122101003',
 'https://book.impress.co.jp/books/1122101004',
 'https://book.impress.co.jp/books/1122101005',
 'https://book.impress.co.jp/books/1122101008',
 'https://book.impress.co.jp/books/1122101009',
 'https://book.impress.co.jp/books/1122101011',
 'https://book.impress.co.jp/books/1122101050',
 'https://book.impress.co.jp/books/1122101054',
 'https://book.impress.co.jp/books/1122101055',
 'https://book.impress.co.jp/books/1121101077',
 'https://book.impress.co.jp/books/1122101047',
 'https://book.impress.co.jp/books/1122101048',
 'https://book.impress.co.jp/books/1122101049',
 'https://book.impress.co.jp/books/1122101001',
 'https://book.impress.co.jp/books/1122101002',
 'https://book.impress.co.jp/books/1122101006',
 'https://book.impress.co.jp/books/1122101007',
 'https://book.impress.co.jp/books/1121101139',
 'https://book.impress.co.jp/books/11221

In [48]:
def get_book_info(book_url):
    # 書籍ページをスクレイピングする準備
    res = requests.get(book_url)
    html_doc = res.text
    soup = BeautifulSoup(html_doc, 'html.parser')
    
    # 書籍情報のブロック
    div_book_detail = soup.find('div', class_='block-book-detail')

    # 書籍名
    book_title = div_book_detail.find('h2')
    # 値段
    book_price = div_book_detail.find('p', class_='module-book-price')
    
    # 発売日・著者
    book_data = {}
    dl_book_data = div_book_detail.find('dl', class_='module-book-data')
    for tag in dl_book_data.find_all(['dt', 'dd']):
        if tag.name == 'dt':
            key = tag.get_text()
        if tag.name  == 'dd':
            book_data[key] = tag.get_text().strip()

    return [
        book_title.get_text(),  # 書籍名
        book_price.get_text(),  # 値段
        book_data['発売日'],  # 発売日
        book_data['著者'],  # 著者
    ]

In [49]:
get_book_info('https://book.impress.co.jp/books/1116101151')

['いちばんやさしいPythonの教本 人気講師が教える基礎からサーバサイド開発まで',
 '2,420円（本体 2,200円＋税10%）',
 '2017/8/10',
 '鈴木 たかのり\u3000著/杉谷 弥月\u3000著/株式会社ビープラウド\u3000著']

In [50]:
import time

book_info_list = []
for book_url in book_urls:
    print('スクレイピング中: ', book_url)
    book_info_list.append(get_book_info(book_url))
    time.sleep(0.1)
print('完了')

スクレイピング中:  https://book.impress.co.jp/books/1121101107
スクレイピング中:  https://book.impress.co.jp/books/1122101010
スクレイピング中:  https://book.impress.co.jp/books/1122101003
スクレイピング中:  https://book.impress.co.jp/books/1122101004
スクレイピング中:  https://book.impress.co.jp/books/1122101005
スクレイピング中:  https://book.impress.co.jp/books/1122101008
スクレイピング中:  https://book.impress.co.jp/books/1122101009
スクレイピング中:  https://book.impress.co.jp/books/1122101011
スクレイピング中:  https://book.impress.co.jp/books/1122101050
スクレイピング中:  https://book.impress.co.jp/books/1122101054
スクレイピング中:  https://book.impress.co.jp/books/1122101055
スクレイピング中:  https://book.impress.co.jp/books/1121101077
スクレイピング中:  https://book.impress.co.jp/books/1122101047
スクレイピング中:  https://book.impress.co.jp/books/1122101048
スクレイピング中:  https://book.impress.co.jp/books/1122101049
スクレイピング中:  https://book.impress.co.jp/books/1122101001
スクレイピング中:  https://book.impress.co.jp/books/1122101002
スクレイピング中:  https://book.impress.co.jp/books/1122101006
スクレイピング中: 

In [51]:
book_info_list

[['いちばんやさしいEC担当者の教本 人気講師が教える新任1年目に身につけたい実務と知識',
  '2,200円（本体 2,000円＋税10%）',
  '2022/9/15',
  '中島 郁\u3000著/南茂 理恵\u3000著'],
 ['【Amazon.co.jp限定】目標を楽しくおしゃれに叶えるオトナの勉強手帳 Study＋Diary2023',
  '1,540円（本体 1,400円＋税10%）',
  '2022/9/15',
  'インプレス手帳編集部\u3000編'],
 ['ふわころことり Schedule Book 2023',
  '1,430円（本体 1,300円＋税10%）',
  '2022/9/15',
  'インプレス手帳編集部\u3000編'],
 ['にゃんこのぷにぷに肉球手帳 2023',
  '1,430円（本体 1,300円＋税10%）',
  '2022/9/15',
  'フェリシモ猫部\u3000監修'],
 ['山野りんりん 猫まみれ手帳 2023',
  '1,430円（本体 1,300円＋税10%）',
  '2022/9/15',
  'フェリシモ猫部\u3000監修'],
 ['季節とともに機嫌よく過ごす暮らし手帳 2023',
  '1,870円（本体 1,700円＋税10%）',
  '2022/9/15',
  'インプレス手帳編集部\u3000編'],
 ['目標を楽しくおしゃれに叶えるオトナの勉強手帳 Study＋Diary2023',
  '1,540円（本体 1,400円＋税10%）',
  '2022/9/15',
  'インプレス手帳編集部\u3000編'],
 ['推しとの思い出を楽しくキロクする 推し活ライフ手帳2023',
  '1,518円（本体 1,380円＋税10%）',
  '2022/9/15',
  'インプレス手帳編集部\u3000編'],
 ['できるポケット Word & Excel 2021 基本＆活用マスターブック Office 2021&Microsoft 365両対応',
  '1,408円（本体 1,280円＋税10%）',
  '2022/9/13',
  '田中\u3000亘\u3000著/羽毛田睦土\u3000著/できるシリーズ編集部\u3000

### 手順パート: 収集した書籍情報を保存する

In [52]:
with open('book_data.tsv', 'w', encoding='utf-8') as f:
    for book_info in book_info_list:
        f.write('\t'.join(book_info) + '\n')

### 手順パート: TSVファイルから書籍を検索する

In [53]:
def book_search(keyword):  # キーワードを受け取る
    results = []  # キーワードが含まれる行を格納する
    with open('book_data.tsv', encoding='utf-8') as f:  # tsvファイルを開く
        for line in f:  # 各行を読み込む
            cols = line.split('\t')  # タブ文字で分割する
            if keyword in cols[0]:  # 書籍名にキーワードが含まれるか
                results.append(line)
    if len(results) > 0:  # 1件以上検索ヒットした場合
        response =  ''.join(results)  # 連結する
    else:  # 検索ヒットしたものがなかった場合
        response = '「{}」ガ含マレル書籍ガ見ツカリマセンデシタ'.format(keyword)
    return response

In [57]:
response = book_search('1500')
print(response)

「1500」ガ含マレル書籍ガ見ツカリマセンデシタ


In [31]:
response = book_search('寿司')
print(response)

「寿司」ガ含マレル書籍ガ見ツカリマセンデシタ
